In [7]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from dash import no_update

# Create app
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True

# Read data
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Extract year from the Date column if needed
df['Year'] = pd.to_datetime(df['Date']).dt.year

# App Layout
app.layout = html.Div(children=[
    html.H1('Automobile Sales Statistics Dashboard', 
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),

    # Outer division starts
    html.Div([
        # First dropdown: Report Type
        html.Div([
            html.Label('Select Report Type:'),
            dcc.Dropdown(
                id='report_type',
                options=[
                    {'label': 'Yearly Statistics', 'value': 'Yearly'},
                    {'label': 'Recession Period Statistics', 'value': 'Recession'}
                ],
                value='Yearly',  # Default value
                placeholder="Select a report type"
            ),
        ], style={'width': '48%', 'display': 'inline-block'}),

        # Second dropdown: Year
        html.Div([
            html.Label('Select Year:'),
            dcc.Dropdown(
                id='year_dropdown',
                options=[{'label': year, 'value': year} for year in sorted(df['Year'].unique())],
                placeholder="Select a year"
            ),
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Br(),

        # Container for all four plots
                html.Div(id='output_container', className='chart-grid', style={'display': 'flex'}, children=[
                    html.Div([], id='plot1'),
                    html.Div([], id='plot2'),
                    html.Div([], id='plot3'),
                    html.Div([], id='plot4'),
                ])

    ])
])

@app.callback(
    Output('year_dropdown', 'disabled'),
    Input('report_type', 'value')
)
def toggle_year_dropdown(report_type):
    if report_type == 'Recession':
        return True  
    else:
        return False  

@app.callback(
    [Output('plot1', 'children'),
     Output('plot2', 'children'),
     Output('plot3', 'children'),
     Output('plot4', 'children')],
    [Input('report_type', 'value'),
     Input('year_dropdown', 'value')]
)
def update_dashboard(report_type, selected_year):
    if report_type == 'Yearly' and selected_year is not None:
        df_year = df[df['Year'] == selected_year]

        # Plot 1: Total Sales by Vehicle Type
        fig1 = px.bar(df_year, x='Vehicle_Type', y='Automobile_Sales', 
                      title=f'Automobile Sales by Vehicle Type in {selected_year}')

        # Plot 2: Advertising Expenditure
        fig2 = px.pie(df_year, names='Vehicle_Type', values='Advertising_Expenditure',
                      title=f'Advertising Expenditure Share by Vehicle Type in {selected_year}')

        # Plot 3: Sales Trend
        fig3 = px.line(df_year, x='Month', y='Automobile_Sales', 
                       title=f'Monthly Sales Trend in {selected_year}')

        # Plot 4: GDP vs Sales
        fig4 = px.scatter(df_year, x='GDP', y='Automobile_Sales', color='Vehicle_Type',
                          title=f'GDP vs Automobile Sales in {selected_year}')

    elif report_type == 'Recession':
        df_rec = df[df['Recession'] == 1]

        # Plot 1: Total Sales by Vehicle Type during Recession
        fig1 = px.bar(df_rec, x='Vehicle_Type', y='Automobile_Sales', 
                      title='Automobile Sales by Vehicle Type during Recession')

        # Plot 2: Advertising Expenditure during Recession
        fig2 = px.pie(df_rec, names='Vehicle_Type', values='Advertising_Expenditure',
                      title='Advertising Expenditure Share by Vehicle Type during Recession')

        # Plot 3: Sales Trend during Recession
        fig3 = px.line(df_rec, x='Year', y='Automobile_Sales', color='Vehicle_Type',
                       title='Yearly Sales Trend during Recession')

        # Plot 4: Unemployment Rate vs Sales
        fig4 = px.scatter(df_rec, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                          title='Unemployment Rate vs Automobile Sales during Recession')
    else:
        return [no_update, no_update, no_update, no_update]

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2),
            dcc.Graph(figure=fig3), dcc.Graph(figure=fig4)]


# Run the app
if __name__ == '__main__':
    app.run(port=8052)
